## **Some Helpful Libraries**
- pip install those not available in your environment

In [3]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

import re

import warnings
warnings.filterwarnings('ignore')


from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **Function to remove Unwanted Letters using Reges**

In [4]:
import re
ip_addr_regex = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\r\n", "", phrase)            # Removing additional line
    phrase = re.sub(r"\n", "", phrase)              # Removing additional line 
    phrase = re.sub(r"\S*@\S*\s?", "", phrase)      # Removing email-addresses 
    phrase = re.sub(r'http\S+', '', phrase)         # Removing website links
    phrase = re.sub(ip_addr_regex, "", phrase)      # Removing IP address link.
    phrase = re.sub(r'[!"\$%&\'()*+,\-.\/:;=#@?\[\\\]^_`{|}~]*', "", phrase)
    phrase = emoji_pattern.sub(r'', phrase)         # Remove Emojis
    
    return phrase

## **Removing Certain Stopwords from English-Stopwords list**

In [5]:
# Removing important words from Stopwords list. ( ‘no’, ‘not’, ‘don’, ‘won’)
# To Remove more keeo adding word to the list
stop_words = stopwords.words('english')
words_to_be_removed = ['no', 'not', 'don', 'won']
for word in words_to_be_removed:
  stop_words.remove(word)

## **Function that can be used to:**
- **Preprocess text**
- **can be passed as analyzer to the CountVectorizer function (for preprocessing and creating BoW etc.**

In [6]:
def textPreprocessing(document):
  # storing nltk_words (english) in a variable
  nltk_words = set(nltk.corpus.words.words())

  # Word Tokenization
  words = [" ".join(nltk.wordpunct_tokenize(title)) for title in document.split()]

  # Selecting Only English Words
  only_eng = [word for word in words if word.lower() in nltk_words or not word.isalpha()]

  # Remove Punctuations
  sentWithoutPunct = ' '.join([char for char in only_eng if char not in string.punctuation])

  # Normalize the data (lowercase)
  wordNormalized = ' '.join([word.lower() for word in sentWithoutPunct.split() if len(word)  > 3 ])

  # Remove Stopwords
  vocabulary = [word for word in wordNormalized.split() if word not in stopwords.words('english')]

  # Apply Stemming
  # Creating an instance of PorterStemmer class, for stemming.
  # stemObject = PorterStemmer() # Could be written outside this Function. Putting here for reference only
  # stem_words = ''.join([stemObject.stem(word) for word in vocabulary])
  # vocab= stem_words.split()


  return vocabulary

## **Using CountVectorizer to create BoW**

In [ ]:
"""
features = X
labels = Y
"""
# Create BOW in SKlearn
wordVector = CountVectorizer(analyzer=textPreprocessing)

#Build the Vocabulary
finalWordVectorVocab = wordVector.fit(features)

# To create BOW
bagOfWords = finalWordVectorVocab.transform(features)

In [ ]:
"""CountVectorizer inbuild text cleaning"""
#Create Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(stop_words=stop_words, #Stopwords
                              lowercase=True, # Normalization
                              token_pattern= r'\b[a-zA-Z]{3,}', #Tokenization
                                strip_accents='unicode',  #Trims all unicode chars which is not a part of eng language
                              )

tfObject = tf_vectorizer.fit_transform(df['content'])
tfObject.shape

In [ ]:
"""Create TF-IDF for the above BoW"""
#Create TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
tfidfObject = tfidfVectorizer.fit_transform(df['content'])
tfidfObject.shape



## **TF-IDF object**

In [ ]:
# Apply TFIDF Algo on BOW to create a feature set
tfidfObject = TfidfTransformer().fit(bagOfWords)  #Calc IDF Values

# Lets create Numeric Feature set
processedFeatures = tfidfObject.transform(bagOfWords)

## **Using ``Vader`` for Sentimental-Analysis**

In [8]:
"""Libraries Required"""
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentimentAnalyser = SentimentIntensityAnalyzer()

"""Code for creating a new column in same dataset with sentiments."""
df['labelFromVADER'] = df['text_column'].apply(lambda review: "positive" if sentimentAnalyser.polarity_scores(review)['compound'] > 0.5 else "negative")

## **For creating ``Latent Dirichlet Allocation (LDA)`` model**

In [ ]:
import pandas as pd
import numpy as np

#Visualizing the LDA model
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
#Apply LDA to create topic clusters for Bag of Words

from sklearn.decomposition import LatentDirichletAllocation
lda_tf = LatentDirichletAllocation(n_components=4 , random_state=0)
lda_tf.fit(tfObject) # as per books

In [ ]:
#---------------------LDAModel,BOW/TFIDF Object,CountVectorizerObject ----------
pyLDAvis.sklearn.prepare(lda_tf,tfObject,tf_vectorizer)